<a href="https://colab.research.google.com/github/Shreejan-git/pytorch-complete-course/blob/main/pytorch_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sklearn
from sklearn.datasets import make_circles
import torch
from torch import nn as nn

In [2]:
n_samples = 1000
X, y = make_circles(n_samples,
                    noise=0.03,
                    random_state=42)

In [5]:
# visualize the dataset in pandas Dataframe
import pandas as pd
circles = pd.DataFrame({"x1":X[:,0],
                        "x2": X[:,1],
                        "label": y

})
circles.sample(5)

,x1,x2,label
598,0.461089,0.601066,1
974,0.417560,-0.887706,0
299,-0.406919,0.911747,0
960,0.954660,-0.332228,0
952,-0.521714,0.595483,1


In [6]:
#checking the input and output shapes and their types
X.shape, y.shape

((1000, 2), (1000,))

In [15]:
print(X[0])
print(type(X)) # X is in numpy array form
print(X.dtype) # numpy float is of 64bit

[0.75424625 0.23148074]
<class 'numpy.ndarray'>
float64


In [14]:
print(f"Shape of the X sample: {X[0].shape}")
print(f"Shape of the label: {y[0].shape}")

Shape of the X sample: (2,)
Shape of the label: ()


In [16]:
# converting the data into the torch tensor.
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)
print(X[0])
print(type(X[0]))
print(X[0].dtype) # torch float is 32 bit.

tensor([0.7542, 0.2315])
<class 'torch.Tensor'>
torch.float32


In [17]:
# Splitting the data into train test set
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Number of X_train dataset: {X_train.shape}')
print(f'Number of X_test dataset: {X_test.shape}')

Number of X_train dataset: torch.Size([800, 2])
Number of X_test dataset: torch.Size([200, 2])


In [19]:
print(f'Type after splitting: {type(X_test)}')

Type after splitting: <class 'torch.Tensor'>


In [21]:
print(X_train.device) # our dataset is in cpu. Later, we need to make it a device agnostic code.

cpu


In [4]:
print(torch.cuda.is_available()) # Checking whether we have a access to a cuda or not.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

False


'cpu'

In [25]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(in_features=2, out_features=8)
        self.linear_2 = nn.Linear(in_features=8, out_features=1)

    def forward(self, X):
        return self.layer_2(self.linear_1(X))

model_0 = BinaryClassification().to(device)
print(model_0)

BinaryClassification(
  (linear_1): Linear(in_features=2, out_features=8, bias=True)
  (linear_2): Linear(in_features=8, out_features=1, bias=True)
)


In [5]:
model_1 = nn.Sequential(
    nn.Linear(2, 8),
    nn.Linear(8,1)
).to(device)
print(model_1)
print()
print(model_1.state_dict())

Sequential(
  (0): Linear(in_features=2, out_features=8, bias=True)
  (1): Linear(in_features=8, out_features=1, bias=True)
)

OrderedDict([('0.weight', tensor([[-0.0394, -0.3803],
        [-0.0453,  0.7034],
        [ 0.5691,  0.2758],
        [ 0.4293, -0.4214],
        [ 0.5507,  0.6977],
        [ 0.0636,  0.0690],
        [ 0.3049,  0.0379],
        [ 0.5353, -0.2839]])), ('0.bias', tensor([ 0.0325, -0.0598, -0.5344, -0.5239, -0.2829, -0.4976, -0.1105, -0.1164])), ('1.weight', tensor([[-0.0386,  0.3137,  0.2049,  0.2944, -0.1277,  0.2848,  0.3183,  0.3225]])), ('1.bias', tensor([-0.0861]))])


In [29]:
for i, j in  model_0.state_dict().items():
    print(f'{i}: {j} \n')

linear_1.weight: tensor([[-0.2068, -0.4273],
        [ 0.4203,  0.2991],
        [ 0.0238,  0.0496],
        [-0.4906,  0.5028],
        [-0.1537, -0.3698],
        [-0.1719,  0.4471],
        [-0.1311,  0.4644],
        [ 0.4775,  0.6708]]) 

linear_1.bias: tensor([-0.4577,  0.5766, -0.0110, -0.6560,  0.5062,  0.5990,  0.3491, -0.2468]) 

linear_2.weight: tensor([[ 0.2358, -0.1084,  0.3169,  0.1293, -0.1937, -0.2196, -0.2333, -0.2822]]) 

linear_2.bias: tensor([0.3166]) 



In [32]:
print(f'Our model (learnable parameters) is in {next(model_0.parameters()).device}')

Our model is in cpu


In [9]:
# Defining the classification based loss functions
# For Binary Classification, we can have 2 different(but same)loss functions.
# 1: nn.BCELoss(), 2: nn.BCEWithLogitLoss()

loss_fn = nn.BCELoss() # requires inputs to have gone through the sigmoid activation function prior to input to BCELoss
loss_fun = nn.BCEWithLogitsLoss() # this has built-in sigmoid activation function.
loss_fun

BCEWithLogitsLoss()

In [7]:
# Defining the optimization
optimizer = torch.optim.Adam(model_1.parameters(), lr=0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [ ]:
# calculate the accuracy
